In [318]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys   
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import time
import numpy as np

In [319]:
taglist = ['avocado','avocados','avocadotoast','wherearetheavocados','avocadooil','avocadolove','avocadolover','avocadoplant','avocadosalad','avocadoporn','avocadotree','avocadolovers','avocadoaddict','aguacate']

In [320]:
driver = webdriver.Chrome('/Users/adriaferrer/IronhackRepos/Avocalypse/Tests/chromedriver')

In [321]:
# Define lists to store post information
 
tag_name_list = []
postids_list = []
post_time_list = []
likes_list = []

# Loop over each hashtag to extract information
for tag in taglist:
    
    for scroll in range(10):
        print(f'This is scroll {scroll}')
        driver.get('https://www.instagram.com/explore/tags/'+str(tag))
        time.sleep(1)
        
        # Extract current hashtag name
        tagname = tag
        
        # Extract all post links from 'explore tags' page
        post_links = []

        if scroll == 0:
            soup = BeautifulSoup(driver.page_source,"lxml")
            recientes = soup.find_all('div', {'class': 'Nnq7C weEfm'})
            recientes = recientes[4:]
            for row in recientes:
                for a in row.find_all('a', href=True):
                    post_links.append(a['href'])            
        else:
            for times in range(scroll):
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(1)
            recientes = driver.find_elements_by_class_name('KC1QD')
            for row in recientes:
                recientes = row.find_elements_by_tag_name('a')
            for a in recientes:
                post_links.append(a.get_attribute('href').strip('https://www.instagram.com'))
            post_links = post_links[9:]

        # Keep link of only 30 most recent posts
        if scroll == 0:
            clean_post_links = [x for x in post_links if x.startswith('/p/')]
        else:
            clean_post_links = ['/p/'+x for x in post_links]
        
        del clean_post_links[30:]

        # Extract the info for each post of the tag
        for postid in range(len(clean_post_links)):

            #Add tagname
            tag_name_list.append(tag)
            driver.get('https://www.instagram.com'+str(clean_post_links[postid]))
            soup = BeautifulSoup(driver.page_source,"lxml")

            #Add post id
            postids_list.append(str(clean_post_links[postid]))

            #Add likes, if any
            likes = soup.find('div', {'class': 'Nm9Fw'})
            if likes == None:
                likes_list.append('0')
            else:
                try:
                    likes_list.append(likes.find('span').text)
                except:
                    likes_list.append('1')

            #Add timestamp
            i = soup.find('time')
            if i == None:
                post_time_list.append(np.nan)
            else:
                post_time_list.append(i['datetime'])
    
driver.quit()

This is scroll 0
This is scroll 1
This is scroll 2
This is scroll 3
This is scroll 4
This is scroll 5
This is scroll 6
This is scroll 7
This is scroll 8
This is scroll 9
This is scroll 0
This is scroll 1
This is scroll 2
This is scroll 3
This is scroll 4
This is scroll 5
This is scroll 6
This is scroll 7
This is scroll 8
This is scroll 9
This is scroll 0
This is scroll 1
This is scroll 2
This is scroll 3
This is scroll 4
This is scroll 5
This is scroll 6
This is scroll 7
This is scroll 8
This is scroll 9
This is scroll 0
This is scroll 1
This is scroll 2
This is scroll 3
This is scroll 4
This is scroll 5
This is scroll 6
This is scroll 7
This is scroll 8
This is scroll 9
This is scroll 0
This is scroll 1
This is scroll 2
This is scroll 3
This is scroll 4
This is scroll 5
This is scroll 6
This is scroll 7
This is scroll 8
This is scroll 9
This is scroll 0
This is scroll 1
This is scroll 2
This is scroll 3
This is scroll 4
This is scroll 5
This is scroll 6
This is scroll 7
This is scroll

In [322]:
len(postids_list)

1794

In [323]:
len(set(postids_list))

474

In [324]:
len(post_time_list)

1794

In [325]:
len(likes_list)

1794

In [326]:
# Add hashtag info to dataframe

tag_df  = pd.DataFrame({'tags': tag_name_list, 'postid': postids_list, 'date': post_time_list, 'likes': likes_list})         

In [328]:
# Check the final dataframe
tag_df.head()
 
# CSV output for hashtag analysis
tag_df.to_csv('hashtag_list.csv')